In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
%pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

In [ ]:
GROQ_KEY="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"

In [ ]:
NEO4J_URI="neo4j+s://41da11af.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="JkZ1FHG9YM90U4vEnLBMkiufOTDwH4rrfJYKUlcMgjY"
AURA_INSTANCEID="41da11af"
AURA_INSTANCENAME="Instance01"

In [ ]:
import os
os.environ['NEO4J_URI'] = NEO4J_URI
os.environ['NEO4J_USERNAME'] = NEO4J_USERNAME
os.environ['NEO4J_PASSWORD'] = NEO4J_PASSWORD


In [ ]:
from langchain_community.graphs import Neo4jGraph

In [ ]:
graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(groq_api_key=GROQ_KEY, model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7be42d1f7850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7be42da6f310>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.documents import Document

In [ ]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query='Virat Kohli').load()

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm,node_properties=True,relationship_properties=True)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True)

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7be42da6c880>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7be42d1f7850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7be42da6f310>, model_name='Gemma2-9b-I

In [ ]:
chain.invoke({"query":"When Elizabeth I was Born"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id:"Elizabeth I"}) RETURN p.born
Full Context:
[{'p.born': '1533-09-07'}]

> Finished chain.


{'query': 'When Elizabeth I was Born', 'result': 'September 7, 1533.  \n'}

In [ ]:
import requests
import json
def generate_answer_with_llm(prompt:str) -> str:
  api_key = "nipqq2mHYIQjJVDsWKXzCB49vGab1wBB"
  url = "https://api.deepinfra.com/v1/openai/chat/completions"
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
  }

  data = {
    "model": "google/gemma-2-27b-it",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ]
  }
  response = requests.post(url, headers=headers, data=json.dumps(data))
  # Check the response
  if response.status_code == 200:
    result = response.json()
    print(result['choices'][0]['message']['content'])
  else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [ ]:
def prompt_formatter(query: str,context) -> str:

  based_prompt = """Based on this Context Items , Please answer the query.
  Query:{query}
  context:{context}
  Answer
  """


  prompt = based_prompt.format(query=query,context=context)

  return prompt

In [ ]:
query = input("Enter your Query:").strip()
result = chain.invoke({"query": query})
print(result)

prompt = prompt_formatter(query=query,context=result)
generate_answer_with_llm(prompt)

Enter your Query:Who Is Virat Khli


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: 'Virat Kohli'}) RETURN p

Full Context:
[{'p': {'birthPlace': 'Delhi', 'occupation': 'Indian cricketer', 'born': '5 November 1988', 'former_captain': 'India national cricket team', 'id': 'Virat Kohli'}}]

> Finished chain.
{'query': 'Who Is Virat Khli', 'result': 'Virat Kohli is an Indian cricketer born in Delhi on 5 November 1988. He is a former captain of the India national cricket team.  \n'}
Virat Kohli is an Indian cricketer born in Delhi on 5 November 1988. He is a former captain of the India national cricket team. 

